In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
## Inputs from user
input_file = 'L2355merged.csv'
num_factors = 5
fixed_loading = '5'
thr = 3.8415

In [3]:
## Utility functions

def get_factors(num_factors, fixed_loading):
    factors_sets = [list(range(1, num_factors+1))]
    u_set = "".join(str(x) for x in factors_sets[0])
    unique_factor_sets = []
    num_sets = len(factors_sets)
    set_counter = 0
    f = []

    children = {}
    parents = {u_set: []}

    while set_counter < num_sets:
        subset = factors_sets[set_counter]
        parent_set = "".join(str(x) for x in subset)
        unique_factor_sets.append(parent_set)

        for indx in range(len(subset)):
            factor_set_pairs = [parent_set]
            one_deg_free_subset = list(np.delete(subset, indx))

            if len(one_deg_free_subset) > 0:
                f.append(one_deg_free_subset)
                str_one_deg_free_subset = "".join(str(x) for x in one_deg_free_subset)
                unique_factor_sets.append(str_one_deg_free_subset)

                if parent_set in children.keys():
                    children[parent_set].append(str_one_deg_free_subset)
                else:
                    children[parent_set] = [str_one_deg_free_subset]

                if str_one_deg_free_subset in parents.keys():
                    parents[str_one_deg_free_subset].append(parent_set)
                else:
                    parents[str_one_deg_free_subset] = [parent_set]

        set_counter += 1

        if set_counter == num_sets:
            factors_sets = f
            num_sets = len(factors_sets)
            set_counter = 0
            f = []

    def get_length(elem):
        return len(elem)

    unique_factor_sets = sorted(list(set(unique_factor_sets)), reverse=True, key=get_length)

    for key in children:
        children[key] = list(set(children[key]))
        
    if fixed_loading is not None:
        for key in children:
            if fixed_loading in key:
                temp = []
                for item in children[key]:
                    if fixed_loading in item:
                        temp.append(item)
                children[key] = temp

    for key in parents:
        parents[key] = list(set(parents[key]))

    return unique_factor_sets, children, parents


def get_chi_square_differences(chi_square, children):
    chi_square_difference = {}

    for key in children:
        for item in children[key]:
            chi_square_difference[(key,item)] = chi_square[key] - chi_square[item]
            
    return chi_square_difference
    

# Recursive forward search implementation
def rec_fs(self, parent):
    if parent is None or chi_square_difference[parent, self] >= thr:
        flag = True
        if parent is not None:
            siblings = list(set(children[parent]) - set([self]))

            for sib in siblings:
                if not chi_square_difference[parent, self] > chi_square_difference[parent, sib]: # reverse comparison
                    flag = False
                
        if flag:
            if self in children.keys():
                self_children = list(set(children[self]))
                for child in self_children:
                    if not chi_square_difference[self, child] < thr: # reverse comparison
                        flag = False
                        
                        if len(child) > 1: # Stop after level FOUR
                            rec_fs(child, self) # recursively call fs for the next level
        
        if flag:
            fr[self] = 1.
            
# Recursive backward search implementation            
def rec_bs(self, child):
    if chi_square_difference[self, child] < thr:
        siblings = list(set(parents[child]) - set([self]))
        
        flag = True
        for sib in siblings:
            if not chi_square_difference[self, child] < chi_square_difference[sib, child]: # reverse comparison
                flag = False
                
        if flag:
            self_parents = parents[self]
            for prnt in self_parents:
                if not chi_square_difference[prnt, self] >= thr: # reverse comparison
                    flag = False
                    
                    rec_bs(prnt, self)
                    
        if flag:
            br[self] = 1.
            

In [4]:
# unit_factors = list(map(str, list(np.arange(1,num_factors))))
# for fixed_loading in unit_factors:

## Create dictionaries containing hierarchies
factors, children, parents = get_factors(num_factors, fixed_loading)

## Read chi square values from file
path = os.path.join('data', input_file)
df = pd.read_csv(path, sep=',')
df = df.loc[:, df.columns.str.startswith('chi')]
df = df.loc[:, ~df.columns.str.contains('NULL')]

column_names = ['chi' + factor for factor in factors]
df = df[column_names] # Re-order columns according to the order in factors
all_simulations_chi_square = df.to_numpy() # Store in np array for easy traversal

fr_columns = None
br_columns = None

fr_values = []
br_values = []
    
## Run search for simulation results
for sim_index in range(len(all_simulations_chi_square)):
    print(f'Search on Simulation {sim_index+1}...')
    
    chi_values = all_simulations_chi_square[sim_index]
    chi_square = {factors[i]: chi_values[i] for i in range(len(factors))}
    chi_square_difference = get_chi_square_differences(chi_square, children)

    # Initialize for current simulation
    fr = {}
    br = {}
    for f in factors:
        fr[f] = 0.
        br[f] = 0.

    # Forward search
    print('Running Forward Search')
    rec_fs(list(parents.keys())[0], None)
    print(fr)
    fr_columns = list(fr.keys()) if fr_columns is None else fr_columns
    fr_values.append(list(fr.values()))

    # Backward Search
    print('Running Backward Search')
    for val in parents[fixed_loading]:
        rec_bs(val, fixed_loading)
    print(br)
    br_columns = list(br.keys()) if br_columns is None else br_columns
    br_values.append(list(br.values()))
    
    print()
    
fr_columns = ['fr' + factor for factor in fr_columns]
forward = pd.DataFrame(fr_values, columns=fr_columns)
br_columns = ['br' + factor for factor in br_columns]
backward = pd.DataFrame(br_values, columns=br_columns)

## Store results in file
if not os.path.exists('results'):
    os.makedirs('results')
    
filename = input_file.replace('.csv','')
    
path = os.path.join('results', f'{filename}_{fixed_loading}_results.csv')
res = pd.concat([forward,backward], axis=1)
res.to_csv(path, index=False)

path = os.path.join('results', f'{filename}_{fixed_loading}_summary.csv')
res_sum = pd.DataFrame(res.sum(axis=0)).T
res_sum.to_csv(path, index=False)

Search on Simulation 1...
Running Forward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 0.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 0.0, '235': 0.0, '124': 0.0, '345': 0.0, '125': 0.0, '134': 0.0, '135': 0.0, '245': 0.0, '12': 0.0, '15': 0.0, '13': 0.0, '45': 0.0, '24': 0.0, '34': 0.0, '23': 0.0, '14': 0.0, '35': 1.0, '25': 0.0, '3': 0.0, '5': 0.0, '2': 0.0, '4': 0.0, '1': 0.0}
Running Backward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 0.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 0.0, '235': 0.0, '124': 0.0, '345': 0.0, '125': 0.0, '134': 0.0, '135': 0.0, '245': 0.0, '12': 0.0, '15': 0.0, '13': 0.0, '45': 0.0, '24': 0.0, '34': 0.0, '23': 0.0, '14': 0.0, '35': 1.0, '25': 0.0, '3': 0.0, '5': 0.0, '2': 0.0, '4': 0.0, '1': 0.0}

Search on Simulation 2...
Running Forward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 0.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 1.0, '235': 0.0, '124': 0.0, '345': 0.0, '125'

Running Backward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 0.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 0.0, '235': 0.0, '124': 0.0, '345': 1.0, '125': 0.0, '134': 0.0, '135': 0.0, '245': 0.0, '12': 0.0, '15': 0.0, '13': 0.0, '45': 0.0, '24': 0.0, '34': 0.0, '23': 0.0, '14': 0.0, '35': 0.0, '25': 0.0, '3': 0.0, '5': 0.0, '2': 0.0, '4': 0.0, '1': 0.0}

Search on Simulation 809...
Running Forward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 1.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 0.0, '235': 0.0, '124': 0.0, '345': 0.0, '125': 0.0, '134': 0.0, '135': 0.0, '245': 0.0, '12': 0.0, '15': 0.0, '13': 0.0, '45': 0.0, '24': 0.0, '34': 0.0, '23': 0.0, '14': 0.0, '35': 0.0, '25': 0.0, '3': 0.0, '5': 0.0, '2': 0.0, '4': 0.0, '1': 0.0}
Running Backward Search
{'12345': 0.0, '1245': 0.0, '1345': 0.0, '2345': 1.0, '1234': 0.0, '1235': 0.0, '234': 0.0, '123': 0.0, '145': 0.0, '235': 0.0, '124': 0.0, '345': 0.0, '125': 0.0, '134': 0.0, '135